In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pybaseball as mlb
import mlbgame

import preprocess
from preprocess import service, model

%load_ext autoreload
%autoreload 2
from jesse import jesse
jesse.SetOptions()

today = datetime.today().strftime('%Y-%m-%d')

### Example of how to generate season dates

In [ ]:
dates = preprocess.get_mlb_dates()
season_days_so_far = dates.get_season_days_so_far()

In [ ]:
for i in range(len(season_days_so_far)):
    d = season_days_so_far[i]
    print('Downloading File FooFile.txt [%d%%]\r'%i, end="")

### Test service + model

In [ ]:
test=preprocess.get_batter_stats()

In [ ]:
# Look for a file
# If it's there, list all its dates
# Pull a list of all the dates I want

In [ ]:
test.Name.value_counts()

In [ ]:
class ReadData:
    def __init__(self):
        self.raw_stats_df = None
    def set_raw_stats_df(self):
        try:
            self.raw_stats_df = _pd.read_csv('../data/csv/raw_batter_stats.csv')
            print('Pulling raw batter stats from existing file...')
        except OSError:
            print('FIRST-TIME-USE INITIALIZE: pulling batter data from pybaseball api...')
            self.raw_stats_df = preprocess.get_raw_batter_stats(year=preprocess.get_prior_years())
    def update_dates_in_raw_stats_df(self):
        dates = preprocess.get_mlb_dates()
        season_days_so_far = dates.get_season_days_so_far()
        dates_we_got = list(self.raw_stats_df.DATE)
        dates_we_dont_got = [d for d in season_days_so_far if d not in dates_we_got]
        if len(dates_we_dont_got) > 0:
            print('Updating batter data to most recent date...')
            print(dates_we_dont_got)
            new_df = _pd.DataFrame(preprocess.service.get_raw_stats(dates_we_dont_got))
            self.raw_stats_df = _pd.concat([self.raw_stats_df, new_df])
        else: print('Batter data is current.')
        self.raw_stats_df.drop_duplicates(inplace=True)
        self.raw_stats_df.sort_values('DATE', inplace=True)
    def get_raw_stats_df(self):
        return self.raw_stats_df

In [ ]:
load = pd.read_csv('../data/csv/raw_batter_stats.csv')
load.shape

In [ ]:
test.shape

### Aggregate all player stats for 2018

In [ ]:
df = pd.DataFrame()
df_lst = []

for d in season_days_so_far:
    try:
        tmp_df = mlb.batting_stats_range(str(d), )
        print(' processing schedule date ', d)
    except:
        continue
    tmp_df['DATE'] = d
    df_lst.append(tmp_df)

In [ ]:
df = pd.concat(df_lst)

In [ ]:
df.shape

### Get key for a player

In [ ]:
mlb.playerid_lookup('gallo', 'joey')

In [ ]:
id = int(mlb.playerid_lookup('gallo', 'joey')['key_mlbam'])
print(id)

### Get season-level stats for a all batters

In [ ]:
mlb.batting_stats_bref(season=2019).head() # entire season
mlb.batting_stats_range(start_dt='2019-03-20', end_dt=today) # specified range

#data = mlb.batting_stats_range("2016-08-24",) 
data = mlb.batting_stats_range("2019-05-28", ) 


In [ ]:
df = pd.DataFrame()
df_lst = []

for d in season_days_so_far:
    try:
        tmp_df = mlb.batting_stats_range(str(d), )
        print(' processing schedule date ',d)
    except:
        continue
    tmp_df['DATE'] = d
    df_lst.append(tmp_df)

In [ ]:
df = pd.concat(df_lst)

In [ ]:
df.sort_values(['Name', 'DATE'], inplace=True)

In [ ]:
df

In [ ]:
today = datetime.today()
days = [today - datetime.timedelta(days=x) for x in range(0, numdays)]

In [ ]:
from datetime import date, timedelta

d1 = date(2008, 8, 15)  # start date
d2 = date(2008, 9, 15)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    print(d1 + timedelta(days=i))



In [ ]:
# Make a list of all days during 2019 baseball season (from beginning to now)
# For each day, pull all data, and append to master dataframe
# data

## Temp

### Get raw stats for a batter

In [ ]:
#gallo_stats = mlb.statcast_batter('2019-04-01', '2019-05-01', id)
gallo_stats = mlb.statcast_batter(start_dt='2019-02-20', end_dt=today, player_id=id)

In [ ]:
gallo_stats[gallo_stats.game_date == '2019-05-01']

In [ ]:
gallo_stats

### Grab all batted ball events for a single day for a single player

In [ ]:
gallo_stats[(gallo_stats.game_date == '2019-05-01') & (~gallo_stats.events.isnull())]

In [ ]:
gallo_stats.events.value_counts()

## What other player sources exist?
* Lahman
** batting stats by year
** updated at the end of each season (current is through 2018)
* fangraps
* baseball reference
* baseball savants


### Do I have to parse the data to one-record-per-game format myself?

### Can I use retrosheet for past years, and use stat_cast for current year?

### How do I define the first game date of the season?

### I should save all player IDs to a table, and append any new players?
